In [25]:
from benchmark_methods import model_output, compute_predictions, logistic_predictions
from proj1_helpers import load_csv_data
import numpy as np

In [19]:
test_data = 'all/test.csv'
test = load_csv_data(test_data)

In [20]:
weights_path = 'all/weights.npy'
weights = np.load(weights_path)

clean_features_path = 'all/clean_features.npy'
clean_features = np.load(clean_features_path)

parameters_path = 'all/parameters.npy'
parameters = np.load(parameters_path)

In [21]:
_, test_data, test_ids, _ = test

In [22]:
out = model_output(test_data, weights, 22, clean_features, parameters)

In [23]:
print(out.shape)

(568238,)


In [28]:
pred = logistic_predictions(out)
print(pred.shape, pred)

(568238,) [0. 0. 0. ... 0. 0. 0.]


## Serializing resulta in csv

In [30]:
import csv

In [42]:
with open('results.csv', 'w', newline='\n', encoding='utf-8') as fp:
    writer = csv.writer(fp)

    for i, id in enumerate(test_ids):
        if not i % 100000: print('Done:', i)
        writer.writerow([id, 's' if pred[i] else 'b'])

Done: 0
Done: 100000
Done: 200000
Done: 300000
Done: 400000
Done: 500000
